In [1]:
import openai
import os
import base64
import requests

In [2]:
openai.api_key = 'lol-get_your_own'
api_key='lol-get-your-own'

In [3]:
textprompt='You are provided with an image, chart, or diagram from an academic paper. Your task is to accurately summarize the information presented. You must adhere strictly to the information given in the image, without introducing any context or additional information not present in the image. Provide an objective and detailed description, using third-person language. Avoid any extrapolation or assumptions beyond what is visually provided.'

In [4]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


In [5]:
image_dir =r'C:\Users\danny\OneDrive - The University of Chicago\DSJ - UCh OneDrive\Summer 2024\Summer Hackathon\alpha\output_images_folder'

In [6]:
dictionary_save_path = 'image_descriptions.json'
processed_images_save_path = 'processed_images.json'

In [7]:
rate_limit_per_minute = 60  # Adjust based on your plan
delay_between_requests = 60 / rate_limit_per_minute

In [8]:
# Initialize or load the dictionary and processed images list
if os.path.exists(dictionary_save_path):
    with open(dictionary_save_path, 'r') as f:
        descriptions = json.load(f)
else:
    descriptions = {}

descriptions

{'attention_Figure_1.png': 'Error: No description available',
 'attention_Figure_2_left.png': 'Error: No description available',
 'attention_Figure_2_right.png': 'Error: No description available'}

In [9]:
if os.path.exists(processed_images_save_path):
    with open(processed_images_save_path, 'r') as f:
        processed_images = json.load(f)
else:
    processed_images = []

In [10]:
processed_images

['attention_Figure_1.png',
 'attention_Figure_2_left.png',
 'attention_Figure_2_right.png']

In [11]:
def get_image_description(base64_image, api_key):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": textprompt
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 300
    }
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response.json()


In [12]:
import time

In [13]:
for image_filename in os.listdir(image_dir):
    if 'ipynb_checkpoints' in image_filename:
        continue
    if image_filename in processed_images:
        continue
    image_path = os.path.join(image_dir, image_filename)
    base64_image = encode_image(image_path)

    response = get_image_description(base64_image, api_key)
    
    # Extract the description from the response
    try:
        description = response['choices'][0]['message']['content']
    except KeyError:
        description = "Error: No description available"

    # Update the dictionary and processed images list
    descriptions[image_filename] = description
    processed_images.append(image_filename)
    
    # Save progress
    with open(dictionary_save_path, 'w') as f:
        json.dump(descriptions, f, indent=4)
    
    with open(processed_images_save_path, 'w') as f:
        json.dump(processed_images, f, indent=4)

    print(image_filename)
    print(description)
    print('')
    time.sleep(1)
print("Processing complete.")

cs224n-2023-lecture11-prompting-rlhf_image10.png
The image contains a text message requesting a suggestion for a 3-course menu for a dinner party. The dinner party is for 6 people who are vegetarian, and the menu must include a chocolate dessert.

cs224n-2023-lecture11-prompting-rlhf_image100.jpg
The image contains text stating, "The most recent Super Bowl was Super Bowl LVI."

cs224n-2023-lecture11-prompting-rlhf_image101.jpg
The image contains text that states the following: "Eagles, who defeated the Kansas City Chiefs by 31-24". 

This indicates that the Eagles won a game against the Kansas City Chiefs with a final score of 31-24.

cs224n-2023-lecture11-prompting-rlhf_image102.jpg
The image contains a blue rectangular button with rounded edges. Inside the button, white text reads, "Who won the superbowl?"

cs224n-2023-lecture11-prompting-rlhf_image103.png
The image contains the text "Bing AI hallucinates the Super Bowl" in large, bold font. This suggests a focus on an incident or sc

In [14]:
response

{'id': 'chatcmpl-9fZZQ0T2lDPn3AMfIsoJh5KHwPLVd',
 'object': 'chat.completion',
 'created': 1719695208,
 'model': 'gpt-4o-2024-05-13',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'The image consists of a horizontal tricolor flag with three sections: a green top section, a white middle section, and a red bottom section. In the center of the flag, the white section features a circular cutout shape that extends slightly into the green and red sections. The flag also displays four vertical red bars of varying heights positioned toward the upper part of the circular cutout, with the tallest bar on the left and the shortest in the middle. The arrangement of these red bars appears in the form of binary code, but exact values are not provided.'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 507,
  'completion_tokens': 112,
  'total_tokens': 619},
 'system_fingerprint': 'fp_4008e3b719'}